In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter

# GPU 자원 사용확인
devices_id = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(
    devices_id
)  # fix bug for `ERROR: all tensors must be on devices[0]`

# Create Tensorboard SummaryWriter instance
writer = SummaryWriter('./summary')
%load_ext tensorboard

# Step 1. Load Dataset
train_dataset = dsets.MNIST(
    root="../data", train=True, transform=transforms.ToTensor(), download=False
)
test_dataset = dsets.MNIST(
    root="../data", train=False, transform=transforms.ToTensor(), download=False
)

# Step 2. Make Dataset Iterable
batch_size = 100
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False
)

In [3]:
# Step 3. Create Model Class
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear1 = torch.nn.Linear(input_dim, 300)
        self.linear2 = torch.nn.Linear(300, int(input_dim / 4))  # 392x196
        self.linear3 = torch.nn.Linear(int(input_dim / 4), output_dim)  # 196x10

    def forward(self, x):
        outputs = self.linear1(x)
        outputs = self.linear2(outputs)
        outputs = self.linear3(outputs)
        return outputs

In [4]:
epochs = 10
input_dim = 784
output_dim = 10
# [test] 만일 MSE을 LOSS 함수로 쓴다면???
# output_dim = 1
lr_rate = 0.01

# Step 4. Instantiate Model Class
model = LogisticRegression(input_dim, output_dim)
if devices_id == type([]):  # -> GPU
    model = nn.DataParallel(model, device_ids=devices_id).cuda()
else:
    model = nn.DataParallel(model, device_ids=[devices_id]).cuda()

# Step 5. Instantiate Loss Class
criterion = torch.nn.CrossEntropyLoss()  # computes softmax and then the cross entropy
# Step 6. Instantiate Optimizer Class
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [5]:
# Step 7. Train Model

for epoch in range(int(epochs)):
    iter = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 28 * 28)
        labels = labels

        images = images.to(devices_id)
        labels = labels.to(devices_id)

        optimizer.zero_grad()
        outputs = model(images)
        # [test] 만일 MSE을 LOSS 함수로 쓴다면???
        # labels = labels.type(torch.FloatTensor)
        # outputs = outputs.type(torch.FloatTensor)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        writer.add_scalar('Train/Loss', loss, epoch)

        iter += 1
        if iter % 200 == 0:
            # calculate Accuracy
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = images.view(-1, 28 * 28)
                images = images.to(devices_id)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                # for gpu, bring the predicted and labels back to cpu fro python operations to work
                predicted = predicted.cpu()
                correct += (predicted == labels).sum()
            accuracy = 100 * correct / total
            print(
                f"[Epoch {epoch}] [Iteration: {i}/{len(train_loader)}] [Loss: {loss.item():.3f}] [Accuracy: {accuracy:.2f}]"
            )

[Epoch 0] [Iteration: 199/600] [Loss: 1.787] [Accuracy: 72.57]
[Epoch 0] [Iteration: 399/600] [Loss: 0.947] [Accuracy: 77.28]
[Epoch 0] [Iteration: 599/600] [Loss: 0.688] [Accuracy: 82.58]
[Epoch 1] [Iteration: 199/600] [Loss: 0.565] [Accuracy: 85.46]
[Epoch 1] [Iteration: 399/600] [Loss: 0.619] [Accuracy: 86.88]
[Epoch 1] [Iteration: 599/600] [Loss: 0.374] [Accuracy: 87.97]
[Epoch 2] [Iteration: 199/600] [Loss: 0.353] [Accuracy: 88.69]
[Epoch 2] [Iteration: 399/600] [Loss: 0.436] [Accuracy: 89.20]
[Epoch 2] [Iteration: 599/600] [Loss: 0.496] [Accuracy: 89.53]
[Epoch 3] [Iteration: 199/600] [Loss: 0.569] [Accuracy: 89.67]
[Epoch 3] [Iteration: 399/600] [Loss: 0.403] [Accuracy: 89.92]
[Epoch 3] [Iteration: 599/600] [Loss: 0.251] [Accuracy: 90.17]
[Epoch 4] [Iteration: 199/600] [Loss: 0.278] [Accuracy: 90.23]
[Epoch 4] [Iteration: 399/600] [Loss: 0.363] [Accuracy: 90.43]
[Epoch 4] [Iteration: 599/600] [Loss: 0.300] [Accuracy: 90.70]
[Epoch 5] [Iteration: 199/600] [Loss: 0.293] [Accuracy:

In [7]:
%tensorboard --logdir=./runs/